https://franky07724-57962.medium.com/using-keras-pre-trained-models-for-feature-extraction-in-image-clustering-a142c6cdf5b1

In [ ]:
import tensorflow as tf
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.cm as cm
import seaborn as sns
from random import randint, sample
import pandas as pd
import pickle
import pathlib
import PIL
from fastprogress.fastprogress import progress_bar
    
from tensorflow.keras import layers
from tensorflow.keras.utils import load_img, image_dataset_from_directory, img_to_array

from keras.applications.vgg16 import preprocess_input, VGG16
from keras.applications.vgg19 import VGG19 
from keras.models import Model

from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

In [ ]:
def load_images(filepath):
    os.chdir(filepath)
    images = []
    
    with os.scandir(filepath) as files:
        for file in files:
            if file.name.endswith('.png'):
                images.append(file.name)
    return images

In [ ]:
images = load_images('../../Final/images')
len(images)

In [ ]:
def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [ ]:
def model_vgg19(filepath):
    model19 = VGG19()
    model19 = Model(inputs=model19.inputs, outputs=model19.layers[-2].output)

    data = {}
    p = filepath

    # loop through each image in the dataset
    for image in progress_bar(images):
        # try to extract the features and update the dictionary
        try:
            feature = extract_features(image,model19) ### CHANGE MODEL HERE AS NEEDED
            data[image] = feature
    # if something fails, save the extracted features as a pickle file (optional)
        except:
            print('except')
            with open(p,'wb') as file:
                pickle.dump(data,file)
    filenames = np.array(list(data.keys()))
    features = np.array(list(data.values()))
    features = features.reshape(-1,4096)
    
    pd.DataFrame(features).to_csv('../model/vgg19_features.csv', index=False)
    pd.DataFrame(filenames).to_csv('../model/vgg19_filenames.csv', index=False)
    
    with open("../model/model_vgg19.pkl", "wb") as f:
        pickle.dump(model_vgg19, f)
    
    return filenames, features

In [ ]:
vgg19_filenames, vgg19_features = model_vgg19('../../Final/images')

In [ ]:
vgg19_features.shape

In [ ]:
pca = PCA(n_components=100, random_state=22)
vgg19_pca_features = pca.fit_transform(vgg19_features)

fig = plt.plot(pca.explained_variance_ratio_)
plt.xlabel("Principal Component")
plt.ylabel("Variance")
plt.title('VGG19 PCA Explained Variance')
plt.savefig('../model/vgg19_pca_variance.png', bbox_inches='tight')
plt.show()

In [ ]:
pca = PCA(n_components=20, random_state=22)
vgg19_pca_features = pca.fit_transform(vgg19_features)
pd.DataFrame(vgg19_pca_features).to_csv('../model/vgg19_pca_features.csv', index=False)


In [ ]:
# this is just incase you want to see which value for k might be the best 
sse = []
list_k = list(range(3, 50))

for k in list_k:
    km = KMeans(n_init = 'auto', n_clusters=k, random_state=22)
    km.fit(vgg19_pca_features)
    
    sse.append(km.inertia_)

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('Sum of squared distance')
plt.savefig('../model/vgg19_kmeans_inertia.png', bbox_inches='tight')
plt.show()

#### Adapted from https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_silhouette_analysis.html#

In [ ]:
X = vgg19_pca_features

from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

range_n_clusters = [10, 20, 30, 40, 50]

kmeans_dict = {}

for n_clusters in range_n_clusters:
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    # The silhouette coefficient can range from -1, 1 but in this example all
    # lie within [-0.1, 1]
    ax1.set_xlim([-0.1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette
    # plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, len(X) + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator
    # seed of 10 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, n_init="auto", random_state=22)
    cluster_labels = clusterer.fit_predict(X)
    pd.DataFrame(cluster_labels).to_csv(f'../model/vgg19_kmeans{n_clusters}.csv', index=False)       
    #pd.DataFrame(clusterer.labels_).to_csv(f'/home/freemal4/Capstone/VGG19/vgg19_kmeans{n_clusters}_v2.csv', index=False)       

    kmeans_dict[n_clusters] = cluster_labels

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed
    # clusters
    silhouette_avg = silhouette_score(X, cluster_labels)
    print(
        "For n_clusters =",
        n_clusters,
        "The average silhouette_score is :",
        silhouette_avg,
    )

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(X, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to
        # cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(
            np.arange(y_lower, y_upper),
            0,
            ith_cluster_silhouette_values,
            facecolor=color,
            edgecolor=color,
            alpha=0.7,
        )

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(
        X[:, 0], X[:, 1], marker=".", s=30, lw=0, alpha=0.7, c=colors, edgecolor="k"
    )

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(
        centers[:, 0],
        centers[:, 1],
        marker="o",
        c="white",
        alpha=1,
        s=200,
        edgecolor="k",
    )

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker="$%d$" % i, alpha=1, s=50, edgecolor="k")

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(
        "Silhouette analysis for KMeans clustering on sample data with n_clusters = %d"
        % n_clusters,
        fontsize=14,
        fontweight="bold",
    )
    plt.savefig(f'../model/vgg19_kmeans{n_clusters}_silhouette.png', bbox_inches='tight')
    plt.show()

In [ ]:
for n_cluster in kmeans_dict.keys():
    print("Number of clusters: ", n_cluster)
    groups = {}
    for file, cluster in zip(images, kmeans_dict[n_cluster]):
        if cluster not in groups.keys():
            groups[cluster] = []
            groups[cluster].append(file)
        else:
            groups[cluster].append(file)
       
    for cluster in range(len(groups)):
        print("Cluster: ", cluster)
        plt.figure(figsize = (25,25));
        # gets the list of filenames for a cluster
    
        files = groups[cluster]
        # only allow up to 20 images to be shown at a time
        if len(files) > 100:
        #print(f"Clipping cluster size from {len(files)} to 10")
            files = sample(files,100)
        # plot each image in the cluster
        for index, file in enumerate(files):
            plt.subplot(10,20,index+1);
            img = load_img(file)
            img = np.array(img)
            plt.imshow(img)
            plt.axis('off')
        
        plt.savefig(f'../model/vgg19_kmeans{n_cluster}_cluster{cluster}.png', bbox_inches='tight')
        plt.show()